In [14]:
# Financial News Retrieval & (draft) Twitter Post Generation

# 1. Retrieving financial news articles for a ticker
# 2. Evaluating news relevance using LLM
# 3. Generating Twitter posts from evaluated news

In [15]:
from urllib.request import urlretrieve

# Download all required scripts
repo = "Jongseok-han/ww_tools/main/news_tool/"
base_url = f"https://raw.githubusercontent.com/{repo}"
files = [
    "financial_news.py",
    "news_evaluation_agent.py",
    "twitter_post_agent_llm.py"
]

for file in files:
    urlretrieve(base_url + file, file)
    print(f"✅ Downloaded {file}")

✅ Downloaded financial_news.py
✅ Downloaded news_evaluation_agent.py
✅ Downloaded twitter_post_agent_llm.py


In [16]:
import requests
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

load_dotenv()

from financial_news import get_financial_news_for_analysis
from news_evaluation_agent import evaluate_news_articles
from twitter_post_agent_llm import generate_twitter_posts_llm

In [17]:
def get_company_name(ticker: str, fmp_api_key: str = None) -> str:
    if fmp_api_key is None:
        fmp_api_key = os.getenv('FMP_API_KEY', '')

    if not fmp_api_key:
        print(f"⚠️  FMP_API_KEY not found. Using ticker '{ticker}' as company name.")
        return ticker

    url = f"https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={fmp_api_key}"
    response = requests.get(url, timeout=10)

    if response.status_code == 200:
        data = response.json()
        if data and len(data) > 0:
            company_name = data[0].get('companyName', ticker)
            print(f"✅ Retrieved company name: {company_name}")
            return company_name

    print(f"⚠️  Failed to get company profile. Using ticker '{ticker}' as company name.")
    return ticker

In [18]:
# ============================================================================
# CONFIGURATION
# ============================================================================

# Select ticker for news retrieval
TICKER = 'NVDA'
COMPANY_NAME = get_company_name(TICKER)

✅ Retrieved company name: NVIDIA Corporation


In [19]:
# ============================================================================
# STEP 1: RETRIEVE FINANCIAL NEWS
# ============================================================================

articles, news_df, summary_text = get_financial_news_for_analysis(
    ticker=TICKER,
    max_results=500,  # maximum number of articles to retrieve
    days_back=1       # Last 1 day (=today)
)

print(f"\n{summary_text}")
print(f"• Unique sources: {news_df['source'].nunique()}")
print(f"• Date range: {news_df['publish_date'].min()} to {news_df['publish_date'].max()}")

✅ Active sources: FMP, Tavily, Finnhub, AlphaVantage

Found 192 articles for NVDA in the last 1 days
• Unique sources: 23
• Date range: 2025-10-14 00:15:00 to 2025-10-15 20:12:43


In [20]:
# ============================================================================
# STEP 2: EVALUATE NEWS RELEVANCE
# ============================================================================

# Customize evaluation criteria (optional)
custom_evaluation_instructions = """
Focus on news that:
- Discusses financial performance, earnings, or revenue
- Covers strategic business decisions or partnerships
- Analyzes market trends or competitive dynamics
- Mentions analyst ratings or price targets
"""

# Evaluate and rank articles
top_news = evaluate_news_articles(
    articles=articles,
    ticker=TICKER,
    company_name=COMPANY_NAME,
    custom_instructions=custom_evaluation_instructions,
    top_k=10,          # Get top 10 most relevant articles
    display_results=True
)

✅ News Evaluation Agent initialized with gemini-2.0-flash

🤖 Evaluating 192 articles for NVDA...
  🔄 Calling Gemini API for evaluation...
  ✅ Evaluation complete: 10 articles selected

📊 TOP 10 MOST RELEVANT NEWS FOR NVDA

#1 📰 NVDA Gets New Bull, ASML Earnings Rally, DLTR Raises Guidance
   ────────────────────────────────────────────────────────────────────────────
   📅 Date: 2025-10-15 10:01 | 📺 Source: youtube.com

   💡 Why This Is Real News:
   This article reports on HSBC upgrading Nvidia to buy from hold and raising its price target to $320 from $200, a new Street high. This is significant news because a major firm sees substantial earnings upside and expansion opportunities for Nvidia, influencing investor sentiment.

   🎯 Key Insights:
   • HSBC's upgrade to 'buy' suggests a strong belief in Nvidia's future performance, potentially driving increased investor demand.
   • The price target of $320 represents a substantial potential gain, making the stock more attractive to inves

In [21]:
# ============================================================================
# STEP 3: GENERATE TWITTER POSTS
# ============================================================================

# Customize tweet generation (optional)
custom_tweet_instructions = """
Create tweets that:
- Use analytical tone
- Include relevant financial data
- Ask thought-provoking questions
- Keep under 280 characters
"""

# Generate tweets using LLM
posts = generate_twitter_posts_llm(
    evaluated_news=top_news[:5],      # Use top 5 articles
    ticker=TICKER,
    company_name=COMPANY_NAME,
    posts_per_article=1,
    custom_instructions=custom_tweet_instructions,
    display_posts=True,
    save_json=False,
    save_image=True,
    image_dir='./tweet_images',
    username='WisdomOwlAI'
)

✅ Twitter Post Agent initialized with gemini-2.0-flash

🐦 Generating Twitter posts for NVDA...
  📝 Generating post 1/5 (Rank #1)...
  📝 Generating post 2/5 (Rank #2)...
  📝 Generating post 3/5 (Rank #3)...
  📝 Generating post 4/5 (Rank #4)...
  📝 Generating post 5/5 (Rank #5)...
  ✅ Generated 5 Twitter posts

🐦 GENERATED TWITTER POSTS FOR $NVDA

📱 POST #1 (Based on News Rank #1)
────────────────────────────────────────────────────────────────────────────────
Tone: Analytical | Hook: Thesis | Length: 216/280 chars

HSBC upgraded $NVDA to 'buy', raising the price target to $320. 

This reflects strong belief in Nvidia's earnings upside and expansion opportunities. Is the market undervaluing its growth potential?

#Nvidia #Stocks

📰 Source Article: youtube.com
📌 Title: NVDA Gets New Bull, ASML Earnings Rally, DLTR Raises Guidance...
🔗 URL: https://www.youtube.com/watch?v=m7lI1YR5Te0
────────────────────────────────────────────────────────────────────────────────

📱 POST #2 (Based on News 